In [1]:
import utils as keu
import pandas as pd

In [10]:
raw_data = pd.read_csv('data/raw_book_info_list.csv',index_col=0)

In [12]:
publisher = raw_data.publisher

In [ ]:
import ast

# publisher 전처리
publisher_to_list = publisher.tolist()

# str type의 list를 다시 list type으로 변환
publisher_to_list = list(map(lambda x : ast.literal_eval(x),publisher_to_list))

# 비어있는 list 제거
publisher_to_list = [l for l in publisher_to_list if l ]

sentence_length = [len(l)for  in publisher_to_list]

print(f'{len(publisher_to_list)} 도서')

3749